In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
from tqdm import tqdm

from sae_lens import SAE
# from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
# from sae_lens import SparseAutoencoder, ActivationsStore

# from steering.eval_utils import evaluate_completions
from steering.utils import normalise_decoder
from steering.patch import generate, scores_2d

# from sae_vis.data_config_classes import SaeVisConfig
# from sae_vis.data_storing_fns import SaeVisData

# import plotly.express as px

torch.set_grad_enabled(False)

/Users/slava/safety/steering_research/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained("gemma-2b", device=device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


Loaded pretrained model gemma-2b into HookedTransformer


In [3]:
hp6 = "blocks.6.hook_resid_post"

sae6, _, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb", # see other options in sae_lens/pretrained_saes.yaml
    sae_id = hp6, # won't always be a hook point
    device = 'cpu'
)

sae6 = sae6.to(device)
normalise_decoder(sae6)

In [4]:
intelligence = sae6.W_dec[10351]   # intelligence and genius
writing = sae6.W_dec[1058]  # writing
anger = sae6.W_dec[1062]  # anger
london = sae6.W_dec[10138]  # London
wedding = sae6.W_dec[8406]  # wedding


In [5]:
# def patch_resid(resid, hook, steering, scale=1, some_text=""):
#     print(some_text)
#     resid[:, :, :] = resid[:, :, :] + steering * scale
#     return resid

# hooks = [
#     (hp6, partial(patch_resid, steering=london, scale=70, some_text="London")),
#     (hp6, partial(patch_resid, steering=wedding, scale=70, some_text="wedding")),
# ]

# generate(model,
#         hooks=hooks,
#         # schedules=[(1, 15), (16, None)],
#         max_new_tokens=35,
#         prompt="I think",
#          )

In [6]:
score_1, score_2, loss, coherence = scores_2d(
    model,
    hp6,
    steering_vectors=[london, wedding],
    prompt="I think", 
    criterions=["Mentions London or London-related things", "Mentions wedding or wedding-related things"],
    scales=[0, 40, 80],
    max_new_tokens=25,
    n_samples=4,
)

loss computation not implemented
loss grid
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
evaluating (0, 0)
['I think I would be a little surprised.\n\nIf I am ever allowed to go back to normal 50% of my work', 'I think it was the second day that I’d been home and I was out doing some groceries when I suddenly got the worst headache', 'I think I’d say the first word, that’s the main key to it! As your speaking, people will probably be', 'I think it\'s a 44 magnum. 7.5" barrel with Hogue rubber grips. It has the adjustable']
evaluating


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


done
evaluating
done
evaluating
done
Mentions London or London-related things 1.0
Mentions wedding or wedding-related things 1.0
Text is coherent, the grammar is correct. 8.0
evaluating (0, 1)
['I think the reason is that you are trying to use a feature (<code>-o</code> ) of the <code>vim</code> command which', 'I think it\'s pretty obvious the most important one is "A."\nThat\'s why I\'ve decided to host the', 'I think every mother at some time, want this feeling. Those feelings to cherish and treasure your child for a lifetime. To cherish for', 'I think of the perfect man in my head all the time. It always follows with, “he’d have to be tall.”']
evaluating
done
evaluating
done
evaluating
done
Mentions London or London-related things 1.0
Mentions wedding or wedding-related things 1.0
Text is coherent, the grammar is correct. 8.0
evaluating (0, 2)
['I think that in today world, there are lots of people who choose to have the same wedding ceremony. That day is a <strong>', "I think a 20